In [32]:
from LibESN.matrix_generator import matrixGenerator
from LibESN.esn import ESN
import numpy as np
import pandas as pd
from pandas import read_csv
import numpy as np
import matplotlib.pyplot as plt
import math


In [33]:
# Load data
data = read_csv(r"C:\Users\zhouq\OneDrive - Nanyang Technological University\FYP\Codes\FYP\Data_transform.csv", header = 0, index_col=0, parse_dates=True, dayfirst=False)

# Select 3 exogenous variables 
data_new = data[['GDP', 'M1', 'M4','M5']]
data_new = data_new.dropna()
display(data_new)

# Split the data into training and testing sets
train_size = int(len(data_new) * 0.8)

GDP_fill_data  = pd.DataFrame(data_new['GDP'])
GDP_fill_data.index = GDP_fill_data.index + pd.offsets.QuarterEnd(0)
GDP_fill_data_train, GDP_fill_data_test = GDP_fill_data[:train_size], GDP_fill_data[train_size:]
display(GDP_fill_data_train)

x_data = pd.DataFrame(data_new.iloc[:, 1:])
x_train, x_test = x_data[:train_size], x_data[train_size:]

display(x_train)

,GDP,M1,M4,M5
Date,,,,
1959-06-30,0.022301,0.012391,0.004168,7.332865
1959-09-30,0.000705,-0.019971,-0.001575,7.316347
1959-12-31,0.002845,0.019589,0.004622,7.281844
1960-03-31,0.022228,0.002654,0.001718,7.204208
1960-06-30,-0.005405,-0.007252,-0.000649,7.145892
...,...,...,...,...
2018-12-31,0.002229,-0.000848,0.001212,7.057013
2019-03-29,0.005960,-0.003236,0.000788,7.086399
2019-06-28,0.007898,-0.001557,0.001012,7.149682


,GDP
Date,
1959-06-30,0.022301
1959-09-30,0.000705
1959-12-31,0.002845
1960-03-31,0.022228
1960-06-30,-0.005405
...,...
2006-09-30,0.001503
2006-12-31,0.008402
2007-03-31,0.002932


,M1,M4,M5
Date,,,
1959-06-30,0.012391,0.004168,7.332865
1959-09-30,-0.019971,-0.001575,7.316347
1959-12-31,0.019589,0.004622,7.281844
1960-03-31,0.002654,0.001718,7.204208
1960-06-30,-0.007252,-0.000649,7.145892
...,...,...,...
2006-09-29,0.000686,0.001264,7.439508
2006-12-29,0.002802,0.000988,7.357986
2007-03-30,0.002561,0.001319,7.286771


In [36]:
# construct individual ESN
# K is size of inputs 
K = len(data_new)
print(int(x_data.shape[1]))

# build single reservoir ESN with reservoir dim = 30, sparsity = 10/30 (33.3%)


esn_a = ESN(
    None,
    smap=np.tanh, 
    A=matrixGenerator((30, 30), dist='sparse_normal', sparsity=10/30, seed=1234), 

    C=matrixGenerator((30, int(x_data.shape[1])), dist='sparse_uniform', sparsity=10/30, seed=1234), 

    zeta=np.zeros((30, 1)), 
    rho=0.5, 
    gamma=1, 
    leak=0,
)

3


In [30]:
from LibESN.mfesn import MFESN
from LibESN.mfesn_fit import mfRidgeFit, mfRidgeCV

# current issue is with the fitting -- the code given is for Multi frequency -->> have to change to single

mfesn_simple = MFESN((esn_a, ))

mfcv = mfRidgeCV().cvHighFreq(
    mfmodel=mfesn_simple,
    train_data=((x_train, ), GDP_fill_data_train,),
    steps=[0,1], # CV at HF for both nowcast and 1-step-ahead
    freqratio=3,
    min_train_size=100,
    test_size=5,
    verbose=True,
)

Lambda = [
    mfcv['cvLambda'][0]['L'],
    mfcv['cvLambda'][1]['L']
]

mfesn_fit = mfesn_simple.fitDirectHighFreq(
    train_data=((data_A_train, ), data_Target_train,), 
    method=mfRidgeFit(Lambda_hf),
    steps=[0,1],
    freqratio=3,
)




AssertionError: All models in a MFESN must be ESN models